# Process text data

# Files that we need to load

- amenities_test_deps.csv
- amenities_test_indeps.csv
- amenities_train_deps.csv
- amenities_train_indeps.csv

- categorical_test_deps.csv
- categorical_test_indeps.csv
- categorical_train_deps.csv
- categorical_train_indeps.csv

- numeric_test_deps.csv
- numeric_test_indeps.csv
- numeric_train_deps.csv
- numeric_train_indeps.csv



In [32]:
%matplotlib inline

In [33]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [34]:
def read_df(filename, valtype):
    df = pd.read_csv(filename, low_memory=False, dtype=valtype)
    return df

In [35]:
from collections import OrderedDict, defaultdict

def create_combined_df(input_dict):
    fdf = pd.DataFrame()
    cols = OrderedDict()
    for k, v in input_dict.items():
        df = read_df('./data/'+k, v)
        colnames = [c for c in df.columns if c not in ['Unnamed: 0']]
        cols[k] = colnames
        fdf = pd.concat([fdf, df], axis=1)
    
    # fdf = fdf.DataFrame(fdf, columns=cols)
    fdf = fdf.drop(['Unnamed: 0'], axis=1)
    return fdf, cols

# Load Training Data

In [36]:
train_inpd = {
    'numeric_train_deps.csv': float, 
    'text_train_deps.csv': str,
}

X_train, cols = create_combined_df(train_inpd)
y_train = pd.read_csv('./data/numeric_train_indeps.csv', low_memory=False)
y_train = y_train.drop(['Unnamed: 0'], axis=1).astype(float)
y_train.sample()

print X_train.shape
print y_train.shape
X_train.sample()
y_train.sample()

(127302, 18)
(127302, 1)


,review_scores_rating
88033,87.0


In [37]:
numeric_cols = cols['numeric_train_deps.csv']
text_cols = cols['text_train_deps.csv']

In [38]:
train_colnames = X_train.columns
dependent_variable = y_train.columns[0]

# Load Test Data

In [39]:
test_inpd = {
    'numeric_test_deps.csv': float, 
    'text_train_deps.csv': str,
}

X_test, cols = create_combined_df(test_inpd)
y_test = pd.read_csv('./data/numeric_test_indeps.csv', low_memory=False)
y_test = y_test.drop(['Unnamed: 0'], axis=1).astype(float)
y_test.sample()

print X_test.shape
print y_test.shape
X_test.sample()
y_test.sample()

(127302, 18)
(62702, 1)


,review_scores_rating
16536,97.0


In [40]:
import numpy as np

def get_rand_idxs(start, end, size, seed=1234):
    np.random.seed(seed)
    return np.random.randint(start, end, size)

In [41]:
def run_prediction(X, y, Xt, yt, model):
    if len(y.shape) == 1:
        y = y.reshape(-1, 1)
    if len(yt.shape) == 1:
        yt = yt.reshape(-1, 1)
    
    res = {'train': {}, 'test': {}}
    
    model.fit(X, y)
    
    train_pred = model.predict(X)
    res['train']['pred'] = train_pred
    
    test_pred = model.predict(Xt)
    res['test']['pred'] = test_pred
    
    
    for name, tup in zip(['train', 'test'], [(y, train_pred), (yt, test_pred)]):
        act, prd = tup[0].ravel(), tup[1].ravel()
        print 'Results for %s' % name
        
        mserr = mean_squared_error(act, prd)
        res[name]['mse'] = mserr
        print '%s Mean Squared Error: %.4f' % (name, mserr)
        
        rsq_score = r2_score(act, prd)
        res[name]['r2'] = rsq_score
        print '%s R-Squared: %.4f' % (name, rsq_score)
        
        plt.scatter(act, prd, alpha=0.5)
        plt.title(name+': predictions vs. actual')
        plt.show()
        plt.scatter(act, act-prd, alpha=0.5)
        plt.title(name+': residuals vs. actual')
        plt.show()
        plt.hist(act-prd, alpha=0.5)
        plt.title(name+': residuals histogram')
        plt.show()
    
    return res

# Impute missing values for Train and Test sets
We will use Median for imputing missing values because it is not as affected by outliers as the Mean.
We will train the Imputer on the training data, and use this to fill the values for both Train and Test sets. 
We will not train a new Imputer on test data. This is important because we do not want to look at the test data when imputing the values.

In [42]:
X_num_train = X_train[numeric_cols]
X_num_test = X_test[numeric_cols]

In [43]:
print X_num_train.shape
print X_num_test.shape

(127302, 12)
(127302, 12)


In [44]:
# Impute missing values using median for both independent and dependent variables
# We don't care for host_has_profile_pic, and host_identity_verified.
# We dropped missing values from our dependent variable earlier, 
# so it won't be affected by imputation here.

import numpy as np
from sklearn.preprocessing import Imputer

# We need the 
num_train_colnames = X_num_train.columns

# Train data first
# axis 0 means impute along columns
train_imp = Imputer(missing_values=np.nan, strategy='median', axis=0, copy=True)
train_imp.fit(X_num_train)
X_train_imp = train_imp.transform(X_num_train)


# Then Test data
num_test_colnames = X_num_test.columns
X_test_imp = train_imp.transform(X_num_test)

print X_train_imp.shape
print X_test_imp.shape

(127302, 12)
(127302, 12)


In [45]:
print len(num_train_colnames)

12


# Handle Outliers
Drop them from Training data, as discussed in the Takeaways section.

In [46]:
# all_columns = list(train_colnames.values) + [dependent_variable]

In [47]:
Train = pd.DataFrame(X_train_imp, columns=num_train_colnames)
Train = pd.concat([Train, X_train[text_cols]], axis=1)
Train = pd.concat([Train, y_train], axis=1)
print Train.shape

(127302, 19)


In [48]:
Test = pd.DataFrame(X_test_imp, columns=num_test_colnames)
Test = pd.concat([Test, X_test[text_cols]], axis=1)
Test = pd.concat([Test, y_test], axis=1)

In [49]:
print Test.shape

(127302, 19)


In [50]:
Test.loc[Test.reviews_per_month > 0]

,bathrooms,bedrooms,beds,cleaning_fee,guests_included,host_listings_count,host_acceptance_rate,host_response_rate,host_has_profile_pic,host_identity_verified,days_delta,reviews_per_month,description,host_about,house_rules,neighborhood_overview,notes,summary,review_scores_rating
0,1.0,1.0,1.0,25.0,1.0,1.0,0.33,1.00,1.0,1.0,757.0,0.53,San Francisco Station AT T Park Safeway,None,None,None,None,San Francisco Station AT T Park Safeway,100.0
1,1.0,1.0,1.0,20.0,1.0,1.0,1.00,1.00,1.0,1.0,1603.0,1.62,a Private Bedroom in a classic Victorian one b...,San Francisco CA resident Midwest born I have ...,We are hoping to encourage families students o...,Geographically Noe Valley really does feel lik...,You will be hard pressed to find a better loca...,a Private Bedroom in a classic Victorian one b...,100.0
2,1.0,1.0,1.0,70.0,1.0,1.0,0.94,0.90,1.0,1.0,1462.0,1.16,STR phone number hidden Private entrance bathr...,None,We expect quiet after 10 00 PM We have a frien...,The West Portal neighborhood has its own charm...,Parking is available with advance notice This ...,STR phone number hidden Private entrance bathr...,89.0
3,1.0,1.0,1.0,70.0,1.0,1.0,1.00,1.00,1.0,1.0,1147.0,0.30,Great place to call a home in SF newly remodel...,None,None,None,None,Great place to call a home in SF newly remodel...,73.0
4,2.0,1.0,1.0,70.0,1.0,3.0,0.96,0.83,1.0,1.0,480.0,0.56,Enjoy the lovely South Hill neighborhood of Sa...,I love San Francisco and all that it has to of...,No parties at any time no loud noise after 10 ...,The neighborhood has a variety of cuisines pre...,This bedroom has twin beds flat screen TV with...,Enjoy the lovely South Hill neighborhood of Sa...,69.0
5,1.0,1.0,1.0,25.0,1.0,1.0,1.00,0.60,1.0,1.0,739.0,1.97,Built 1931 the 3BR 2 5BA is elegant but family...,I m the proud father of two girls and husband ...,Please treat this house like your home,The neighborhood has everything you d expect i...,We are 1 mile from the beach The beach is seve...,Built 1931 the 3BR 2 5BA is elegant but family...,100.0
6,1.0,1.0,1.0,50.0,1.0,1.0,1.00,1.00,1.0,1.0,1536.0,0.03,Hi Welcome Gorgeous unobstructed views of the ...,My name is Maryam and I have been living in Sa...,NaN,None,Please note for groups of more than two I requ...,Hi Welcome Gorgeous unobstructed views of the ...,100.0
7,1.5,3.0,5.0,150.0,6.0,1.0,1.00,1.00,1.0,1.0,1827.0,5.31,Legal SF City approved one bedroom one bathroo...,Hello We are a family of three living in beaut...,Thank you in advance for respecting our house ...,Just a few blocks from Golden Gate Park and UC...,San Francisco Short Term Residential Rental Ce...,Legal SF City approved one bedroom one bathroo...,96.0
8,1.0,1.0,1.0,70.0,2.0,1.0,1.00,1.00,1.0,1.0,1625.0,0.04,A modern studio off a secluded garden court wi...,I m an architect in San Francisco with my own ...,NO SMOKING PLEASE inside or in the courtyard N...,Our Mission neighborhood is gritty and transit...,COMMUNITY A portion of the income from the Mis...,A modern studio off a secluded garden court wi...,100.0
9,1.0,1.0,2.0,75.0,2.0,1.0,1.00,1.00,1.0,1.0,1890.0,0.44,Enjoy 2 private bedrooms in a beautiful home l...,I m a California native and have lived in San ...,1 No smoking please 2 No additional pets 2 won...,The Marina District has an active and friendly...,I really do love to make everyone s stay in Sa...,Enjoy 2 private bedrooms in a beautiful home l...,100.0


In [51]:
def drop_outliers(t):
    # Drop outliers from dependent variable
    h = 0
    if 'review_scores_value' == dependent_variable:
        h = 3
    elif 'review_scores_rating' == dependent_variable:
        h = 80
    t = t.loc[t[dependent_variable] >= h]
    
    # Remove reviews_per_month == 0
    t = t.loc[t.reviews_per_month > 0]

    # 1. Remove 'bathrooms' >= 5, convert to int
    t = t.loc[t.bathrooms < 5]
    t.bathrooms = t.bathrooms.astype(int)

    # 2. Remove 'bedrooms' > 5, convert to int
    t = t.loc[t.bedrooms <= 5]
    t.bedrooms = t.bedrooms.astype(int)

    # 3. Remove 'beds' >= 8, convert to int
    t = t.loc[t.beds < 8]
    t.beds = t.beds.astype(int)

    # 4. Remove 'cleaning_fee' > 400
    t = t.loc[t.cleaning_fee <= 400]

    # 5. Remove 'guests_included' > 8
    t = t.loc[t.guests_included <= 8]
    t = t.loc[t.guests_included > 0]
    t.guests_included = t.guests_included.astype(int)

    # 6. Remove 'host_listings_count' > 100
    t = t.loc[t.host_listings_count <= 100]
    # Remove'host_listings_count' <= 0
    t = t.loc[t.host_listings_count > 0]
    t.host_listings_count = t.host_listings_count.astype(int)
    
    

    # 7. Remove 'host_acceptance_rate' == 0.
    t = t.loc[t.host_acceptance_rate > 0.]

    # 8. Remove 'host_response_rate' == 0.
    t = t.loc[t.host_acceptance_rate > 0.]

    # 9. Remove 'reviews_per_month' > 12.
    t = t.loc[t.reviews_per_month <= 12]
    
    return t

In [52]:
Train = drop_outliers(Train)
Test = drop_outliers(Test)

In [53]:
np.unique(Train.review_scores_rating)

array([ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,
        91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.])

In [54]:
np.unique(Test.review_scores_rating)

array([ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,
        91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.])

# Separate the Dependent and Independet Variables

In [55]:
# Train data
y_train = Train[dependent_variable]
print y_train.shape

X_train = Train.drop([dependent_variable], axis=1)
print X_train.shape

# Test data
y_test = Test[dependent_variable]
print y_test.shape

X_test = Test.drop([dependent_variable], axis=1)
print X_test.shape

(119615,)
(119615, 18)
(58911,)
(58911, 18)


# Write back all data to csv

In [56]:
# Write numeric data

In [57]:
y_train = pd.DataFrame(y_train, columns=[dependent_variable])
y_test = pd.DataFrame(y_test, columns=[dependent_variable])

In [58]:
# Write text data

In [59]:
X_train_text = X_train[text_cols]
X_test_text = X_test[text_cols]

In [60]:
X_train_text.to_csv('./data/fin_text_train_deps.csv', encoding='utf8')
X_test_text.to_csv('./data/fin_text_test_deps.csv', encoding='utf8')

In [61]:
# Write amenities columns

In [62]:
print (X_train_text.shape)
print (X_test_text.shape)

(119615, 6)
(58911, 6)
